In [1]:
import utils
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import scipy.stats as st

[nltk_data] Downloading package punkt to /Users/Sanna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = utils.read_data(utils.DATA)
df.set_index('articleId', inplace=True)
df = df.fillna('')
df.head()

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,...,city,text_preprocessed,nr_tokens,mean_token_length,nr_tokens_teaser,nr_tokens_titelH1,nr_sentences,mean_sentence_length,avgTimeOnPage/wordcount,pageviews-exits
articleId,,,,,,,,,,,,,,,,,,,,,
48620281,21,7,12,7,1012,,112.444444,42.857143,33.333333,33.333333,...,München/Stuttgart,Frische Luft und Bewegung: Diese Kombination r...,796,5.359296,29,9,52.0,15.307692,0.146222,9
48620381,19,6,11,5,1484,,185.500000,42.105263,31.578947,26.315789,...,Berlin/Frankfurt/Main,"Der Wecker klingelt, aufstehen! Doch gerade im...",452,5.938053,33,8,28.0,16.142857,0.420635,8
48622639,2,2,2,2,0,,0.000000,0.000000,100.000000,100.000000,...,Berlin,Eltern auf der Suche nach einem guten Babyphon...,396,5.848485,30,7,26.0,15.230769,0.000000,0
48623085,32,9,20,9,974,,81.166667,37.500000,28.125000,28.125000,...,Berlin,Spülmaschinentabs sollen kleine Alleskönner se...,367,5.594005,30,7,23.0,15.956522,0.235266,12
48623259,24,2,7,2,3797,,223.352941,70.833333,8.333333,8.333333,...,Berlin,Make-up hat heutzutage einen Zweck: Es soll da...,183,5.622951,22,8,8.0,22.875000,1.227214,17


In [126]:
df.loc[48738503]

pageviews                                                                 21
entrances                                                                  5
exits                                                                     10
bounces                                                                    5
timeOnPage                                                              3243
conversions                                                                 
avgTimeOnPage                                                        294.818
stickiness                                                            52.381
entranceRate                                                         23.8095
bounceRate                                                           23.8095
kategorie                                                           ratgeber
rubrik                                                            gesundheit
subRubrik                                                                   

In [3]:
#creating train, dev, test
RANDOM_SEED = 123
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_dev, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED, shuffle=True)
print(df_train.shape, df_dev.shape, df_test.shape)

(712, 38) (89, 38) (90, 38)


In [100]:
train_ids, dev_ids, test_ids = df_train.index, df_dev.index, df_test.index

In [104]:
# zeigt mean avgTimeOnPage/wordcount oder stickiness aufgespalten nach subject-Kategorien
# vielleicht besser nur anhand des train-sets?

subjects = utils.get_set_of_meta_cat('subject')
subject_mean = {}

for s in subjects:
    tmp = utils.get_articles_where(df=df, meta_cat = 'subject', label = s)
    mean_t = np.mean(tmp['avgTimeOnPage/wordcount'])
    mean_s = np.mean(tmp['stickiness'])
    mean_p = np.mean(tmp['pageviews'])
    subject_mean[s] = {'mean_t' : mean_t,
                       'mean_s' : mean_s,
                       'mean_p' : mean_p}
print(subject_mean)

{'Arbeit': {'mean_t': 0.3953509819250898, 'mean_s': 38.490602150192146, 'mean_p': 106.14035087719299}, 'Automobilindustrie': {'mean_t': 0.2312182741116751, 'mean_s': 90.9090909090909, 'mean_p': 11.0}, 'Ernährung': {'mean_t': 0.4674141111785935, 'mean_s': 68.10580123944732, 'mean_p': 11.365853658536585}, 'Essen und Trinken': {'mean_t': 0.43383366881380814, 'mean_s': 67.24310776942355, 'mean_p': 58.0}, 'Familie': {'mean_t': 0.4574435936660513, 'mean_s': 59.04797988602787, 'mean_p': 22.132075471698112}, 'Finanzen': {'mean_t': 0.5783040420314774, 'mean_s': 46.43029617053386, 'mean_p': 40.2}, 'Freizeit': {'mean_t': 0.14622164427105852, 'mean_s': 42.8571428571429, 'mean_p': 21.0}, 'Gesundheit': {'mean_t': 0.3804809022301715, 'mean_s': 59.59361666709484, 'mean_p': 38.41935483870968}, 'Immobilien': {'mean_t': 0.39322307279286167, 'mean_s': 65.02493283383171, 'mean_p': 13.022598870056497}, 'Kosmetik': {'mean_t': 1.2272139625080822, 'mean_s': 70.8333333333333, 'mean_p': 24.0}, 'Medizin': {'mean_

## using subject label matrix as features

In [105]:
df_subject = utils.get_meta_cat_file('subject')

In [106]:
df_subject.head()

,Arbeit,Automobilindustrie,Ernährung,Essen und Trinken,Familie,Finanzen,Freizeit,Gesundheit,Immobilien,Kosmetik,...,Recht,Sport,Steuern,Technik,Tier,Tourismus,Umwelt,Verbraucher,Verkehr,Wohnen
articleId,,,,,,,,,,,,,,,,,,,,,
48620281,0,0,0,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
48620381,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
48622639,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
48623085,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,1
48623259,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [118]:
X_train = np.array(df_subject.loc[train_ids])
X_dev = np.array(df_subject.loc[dev_ids])
X_test = np.array(df_subject.loc[test_ids])

In [119]:
X_train.shape, X_dev.shape, X_test.shape

((712, 22), (89, 22), (90, 22))

In [76]:
print(dev_ids[0])
print(df_subject.loc[49200513])
print(X_dev[0])
print(df.loc[49200513]['subject'])

49200513
Arbeit                1
Automobilindustrie    0
Ernährung             0
Essen und Trinken     0
Familie               0
Finanzen              0
Freizeit              0
Gesundheit            0
Immobilien            0
Kosmetik              0
Medizin               0
Mode                  0
Recht                 0
Sport                 0
Steuern               0
Technik               0
Tier                  0
Tourismus             0
Umwelt                0
Verbraucher           0
Verkehr               0
Wohnen                0
Name: 49200513, dtype: int64
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Arbeit


In [153]:
# define the target labels
#target = 'timeOnPage'
#target =  'pageviews'
#target = 'avgTimeOnPage'
#target = 'stickiness'
target = 'avgTimeOnPage/wordcount'
#target = 'bounceRate'
#target = 'stickiness'

y_train = np.array(df_train[target])
y_dev = np.array(df_dev[target])
y_test = np.array(df_test[target])

In [154]:
y_train.shape, y_dev.shape, y_test.shape

((712,), (89,), (90,))

In [155]:
from sklearn.linear_model import Ridge, LinearRegression, SGDRegressor

In [156]:
model = Ridge()

In [157]:
model.fit(X_train, y_train)

Ridge()

In [158]:
# predict for dev set
pred_dev = model.predict(X_dev)

In [159]:
# postprocessing: replace negative values with 0 (better way? can I give that hint to the model?)
pred_dev[pred_dev < 0] = 0
pred_dev

array([0.37896497, 0.40703084, 0.2609573 , 0.37896497, 0.55304542,
       0.40703084, 0.39983963, 0.38336297, 0.47509983, 0.39983963,
       0.38336297, 0.38336297, 0.40703084, 0.39983963, 0.39983963,
       0.39983963, 0.38336297, 0.42971193, 0.38336297, 0.61140923,
       0.55304542, 0.42971193, 0.37896497, 0.42971193, 0.37896497,
       0.40703084, 0.42971193, 0.37896497, 0.40703084, 0.37896497,
       0.38336297, 0.55304542, 0.38336297, 0.40703084, 0.37896497,
       0.39983963, 0.39983963, 0.40703084, 0.40703084, 0.38336297,
       0.40703084, 0.38336297, 0.39983963, 0.42971193, 0.42971193,
       0.37896497, 0.38336297, 0.40703084, 0.38336297, 0.39983963,
       0.40703084, 0.37896497, 0.40703084, 0.38336297, 0.42971193,
       0.38336297, 0.37896497, 0.2609573 , 0.40703084, 0.38336297,
       0.42971193, 0.38336297, 0.37896497, 0.2609573 , 0.47509983,
       0.40703084, 0.47509983, 0.37896497, 0.38336297, 0.47509983,
       0.38336297, 0.39983963, 0.40703084, 0.38336297, 0.47509

In [160]:
np.array(y_dev)

array([0.42690099, 0.28148148, 0.07051071, 0.02313984, 0.        ,
       0.27526725, 6.77272727, 0.49187853, 0.12149533, 0.22771318,
       0.        , 0.17497231, 0.18918919, 0.2461414 , 0.04403131,
       0.48870056, 0.10939511, 0.70605442, 0.39230104, 0.56643404,
       3.03797468, 0.2893401 , 1.10586481, 0.79515829, 0.81243243,
       0.24804382, 0.35549738, 0.47140039, 0.30449302, 0.05964215,
       0.29310345, 0.34850949, 0.28596994, 0.31992188, 0.46935123,
       0.2368254 , 0.30051743, 0.0953405 , 0.37102804, 0.17074742,
       0.06126126, 0.06223176, 0.09278351, 0.05423332, 0.27622135,
       0.11051693, 0.16338583, 0.19591261, 0.14759887, 0.08545136,
       0.13193018, 1.81504065, 0.33580705, 0.59207819, 1.64021164,
       0.42718447, 0.22326454, 0.17383933, 0.18854962, 0.84572411,
       0.10852713, 0.66248257, 0.        , 2.43774574, 0.83169935,
       0.10606061, 0.29840637, 0.11659389, 4.42944785, 0.75321888,
       0.04223602, 0.        , 0.20930233, 0.17142857, 0.51876

In [161]:
st.pearsonr(pred_dev, y_dev)

(0.033667957411586576, 0.7541123118274932)

In [162]:
model.predict(X_dev[:1])

array([0.37896497])

## tatsächlich einfach den Mittelwert der Subjects vorhersagen (bei denen, die mehrere haben, wiederum Mittelwert?)

In [163]:
df_subject = utils.get_meta_cat_file('subject')

In [164]:
df_subject

,Arbeit,Automobilindustrie,Ernährung,Essen und Trinken,Familie,Finanzen,Freizeit,Gesundheit,Immobilien,Kosmetik,...,Recht,Sport,Steuern,Technik,Tier,Tourismus,Umwelt,Verbraucher,Verkehr,Wohnen
articleId,,,,,,,,,,,,,,,,,,,,,
48620281,0,0,0,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
48620381,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
48622639,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
48623085,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,1
48623259,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51562817,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
51564503,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
51564511,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
subjects = utils.get_set_of_meta_cat('subject')

subject_mean_train = {}
for s in subjects:
    #print(s)
    ids_candidates = df_subject[df_subject[s] == 1].index # take the ones where s == 1
    #print(len(ids_candidates))
    ids = [ ID for ID in ids_candidates if ID in train_ids] # theirof just take the ones in train
    #print(len(ids))
    
    mean_t = np.mean(df.loc[ids]['avgTimeOnPage/wordcount'])
    mean_s = np.mean(df.loc[ids]['stickiness'])
    mean_p = np.mean(df.loc[ids]['pageviews'])
    subject_mean_train[s] = {'mean_t' : mean_t,
                             'mean_s' : mean_s,
                             'mean_p' : mean_p}
print(subject_mean_train)

{'Arbeit': {'mean_t': 0.37619595447595433, 'mean_s': 38.88707093902196, 'mean_p': 91.52127659574468}, 'Automobilindustrie': {'mean_t': 0.2312182741116751, 'mean_s': 90.9090909090909, 'mean_p': 11.0}, 'Ernährung': {'mean_t': 0.4740111189699141, 'mean_s': 70.7095000088689, 'mean_p': 11.430769230769231}, 'Essen und Trinken': {'mean_t': 0.3134978229317856, 'mean_s': 82.10526315789471, 'mean_p': 95.0}, 'Familie': {'mean_t': 0.4272728128837668, 'mean_s': 58.85934927692625, 'mean_p': 22.37037037037037}, 'Finanzen': {'mean_t': 0.581271541318767, 'mean_s': 45.33625730994152, 'mean_p': 19.25}, 'Freizeit': {'mean_t': 0.14622164427105852, 'mean_s': 42.8571428571429, 'mean_p': 21.0}, 'Gesundheit': {'mean_t': 0.4032563624659454, 'mean_s': 59.7165516363176, 'mean_p': 44.837606837606835}, 'Immobilien': {'mean_t': 0.3808411340654103, 'mean_s': 65.2343099313056, 'mean_p': 11.957446808510639}, 'Kosmetik': {'mean_t': nan, 'mean_s': nan, 'mean_p': nan}, 'Medizin': {'mean_t': 0.28342828245298957, 'mean_s': 

In [168]:
# Vorhersage für dev_Set
pred_dev_mean = []
for ID in dev_ids:
    #print(ID)
    #print(df_subject.loc[ID])
    subjects_ID = [ c for c in df_subject.columns if df_subject.loc[ID][c] == 1 ]
    #print(subjects_ID)
    mean_mean_t = np.mean( [subject_mean_train[s]['mean_t'] for s in subjects_ID] )
    #print(mean_mean_t)
    pred_dev_mean.append(mean_mean_t)
    

In [169]:
pred_dev_mean

[0.37619595447595433,
 0.4032563624659454,
 0.2520830410035817,
 0.37619595447595433,
 0.5555591437331113,
 0.4032563624659454,
 0.39876383248566477,
 0.3808411340654103,
 0.4740111189699141,
 0.39876383248566477,
 0.3808411340654103,
 0.3808411340654103,
 0.4032563624659454,
 0.39876383248566477,
 0.39876383248566477,
 0.39876383248566477,
 0.3808411340654103,
 0.4272728128837668,
 0.3808411340654103,
 0.581271541318767,
 0.5555591437331113,
 0.4272728128837668,
 0.37619595447595433,
 0.4272728128837668,
 0.37619595447595433,
 0.4032563624659454,
 0.4272728128837668,
 0.37619595447595433,
 0.4032563624659454,
 0.37619595447595433,
 0.3808411340654103,
 0.5555591437331113,
 0.3808411340654103,
 0.4032563624659454,
 0.37619595447595433,
 0.39876383248566477,
 0.39876383248566477,
 0.4032563624659454,
 0.4032563624659454,
 0.3808411340654103,
 0.4032563624659454,
 0.3808411340654103,
 0.39876383248566477,
 0.4272728128837668,
 0.4272728128837668,
 0.37619595447595433,
 0.3808411340654103

In [170]:
st.pearsonr(pred_dev_mean, y_dev)

(0.03285113970521218, 0.759895229280333)

Kommentar
* sowohl beim direkten Trainieren anhand der subject label Matrix als auch beim einfachen Vorhersagen der Mittelwerte anhand des (oder der) Subject(s) ist nicht sooo toll (aber immerhin bei diesen beiden Versionen mehr oder weniger gleich). Bei stickiness funktioniert es einigermaßen gut (r=0.4)